In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
parcels = gpd.read_file('Tax_Parcels_madison.geojson')

In [3]:
narrowed_parcels = parcels.loc[:,['OBJECTID', 'Parcel', 'XRefParcel', 'Address', 'PropertyClass', 'PropertyUse',
                                  'AreaName', 'Bedrooms', 'CurrentTotal', 'NetTaxes', 'TotalTaxes', 'LotSize', 
                                  'Zoning1', 'LotDepth', 'LotWidth', 'LotType1', 'LotType2','geometry']]


In [5]:
narrowed_parcels["MostCommonPropertyClass"] = (narrowed_parcels.groupby("XRefParcel")["PropertyClass"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonPropertyUse"] = (narrowed_parcels.groupby("XRefParcel")["PropertyUse"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonAreaName"] = (narrowed_parcels.groupby("XRefParcel")["AreaName"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonLotType1"] = (narrowed_parcels.groupby("XRefParcel")["LotType1"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonLotType2"] = (narrowed_parcels.groupby("XRefParcel")["LotType2"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels["MostCommonZoning1"] = (narrowed_parcels.groupby("XRefParcel")["Zoning1"]
                        .transform(lambda x: x.value_counts().index[0]))

narrowed_parcels_grouped = narrowed_parcels.groupby(["XRefParcel","geometry", "MostCommonPropertyClass", 
                  "MostCommonPropertyUse", "MostCommonAreaName", "MostCommonLotType1", "MostCommonLotType2", 
                  "MostCommonZoning1"], sort=False, as_index=False).agg(
    {"Address":"min", "Bedrooms":"sum", "CurrentTotal":"sum", "NetTaxes":"sum","TotalTaxes":"sum", 
     "LotSize":"max","AreaName":"max"})

narrowed_parcels_grouped = gpd.GeoDataFrame(narrowed_parcels_grouped, geometry='geometry')

narrowed_parcels_grouped['taxes_per_sq_foot'] = narrowed_parcels_grouped['TotalTaxes']/narrowed_parcels_grouped['LotSize']
#narrowed_parcels_grouped.explore("taxes_per_sq_foot", cmap="Blues",tooltip=False, popup=True, tiles='Stamen Toner' )

In [12]:
narrowed_parcels_grouped.loc[narrowed_parcels_grouped["Address"].str.startswith('300 S Bedford', na=False)]

,XRefParcel,geometry,MostCommonPropertyClass,MostCommonPropertyUse,MostCommonAreaName,MostCommonLotType1,MostCommonLotType2,MostCommonZoning1,Address,Bedrooms,CurrentTotal,NetTaxes,TotalTaxes,LotSize,AreaName,taxes_per_sq_foot
12463,070923403095,"POLYGON ((-89.38928 43.06629, -89.38990 43.065...",Commercial,Condominium -other,No Area Name Given,5 - Other,0 - No Exception,PD,300 S Bedford St,0,27262000,598072.33,598072.33,1.0,No Area Name Given,598072.33


In [16]:
'''
with further examination, the data for this location is just bad, and is actually bad in both the city and county 
property databases!  I calculated the area (142,040 sq ft) and then we can just inject the correct values.
'''
narrowed_parcels_grouped.at[12463,'LotSize'] = 142040
narrowed_parcels_grouped.at[12463,'taxes_per_sq_foot'] = narrowed_parcels_grouped.at[12463,'TotalTaxes']/narrowed_parcels_grouped.at[12463,'LotSize']

narrowed_parcels_grouped.at[12463,'LotSize']


142040.0

In [17]:
taxes_top = narrowed_parcels_grouped.sort_values(by='taxes_per_sq_foot',ascending=False)
taxes_top.head(20)


,XRefParcel,geometry,MostCommonPropertyClass,MostCommonPropertyUse,MostCommonAreaName,MostCommonLotType1,MostCommonLotType2,MostCommonZoning1,Address,Bedrooms,CurrentTotal,NetTaxes,TotalTaxes,LotSize,AreaName,taxes_per_sq_foot
20074,070924213071,"POLYGON ((-89.37866 43.07430, -89.37884 43.074...",Residential,Condominium,Condo Prkng & Storage in Area 5924,5 - Other,0 - No Exception,PD,137 E Wilson St,105,39813300,856993.41,856993.41,13622.00,Notation Parcels,62.912451
57259,070914425090,"POLYGON ((-89.38587 43.07526, -89.38579 43.075...",Residential,Condominium,Condo Prkng & Storage in Area 5923,5 - Other,0 - No Exception,PD,10 W Mifflin St Unit 100,74,41032900,892463.25,898589.12,16048.00,Notation Parcels,55.993838
41798,070913332139,"POLYGON ((-89.38398 43.07694, -89.38384 43.076...",Residential,Condominium,Condominium Parking in Area 5925,5 - Other,0 - No Exception,PD,120 E Mifflin St,122,38853300,887114.63,892796.15,16520.00,Notation Parcels,54.043350
8732,070923112191,"POLYGON ((-89.38891 43.07286, -89.38983 43.072...",Residential,Condominium,Condo Highrise in Res Area 24,5 - Other,0 - No Exception,PD,321 W Mifflin St Unit P383,577,122053900,2692546.91,2692735.54,63653.00,Notation Parcels,42.303356
37947,070924209046,"POLYGON ((-89.38387 43.07264, -89.38410 43.072...",Commercial,Condominium -other,No Area Name Given,5 - Other,0 - No Exception,PD,117 S Hamilton St,0,29290000,657913.24,665257.49,16008.50,No Area Name Given,41.556516
8759,070923117026,"POLYGON ((-89.38731 43.07159, -89.38774 43.071...",Commercial,Hotel,No Area Name Given,2 - Irregular,0 - No Exception,PD,333 W Washington Ave,0,13900000,446124.31,446124.31,10837.00,No Area Name Given,41.166772
45708,070923110062,"POLYGON ((-89.38531 43.07348, -89.38542 43.073...",Residential,Condominium,Condo Highrise in Res Area 24,5 - Other,0 - No Exception,DC,11 S Fairchild St,125,44273500,983356.28,992773.78,24146.36,Notation Parcels,41.114842
41301,070923201308,"POLYGON ((-89.39453 43.07395, -89.39480 43.073...",Commercial,348 unit Apartment,No Area Name Given,2 - Irregular,1 - Corner,UMX,432 W Gorham St,0,76700000,1723173.48,1731039.61,42936.00,No Area Name Given,40.316741
37657,070913331206,"POLYGON ((-89.38227 43.07691, -89.38263 43.076...",Commercial,Hotel,No Area Name Given,2 - Irregular,0 - No Exception,PD,1 N Webster St,0,15660000,502517.58,502517.58,14530.00,No Area Name Given,34.584830
14410,070923114270,"POLYGON ((-89.39245 43.07288, -89.39281 43.072...",Commercial,160 unit Apartment,No Area Name Given,1 - Regular,0 - No Exception,UMX,433 W Johnson St,0,25290000,568122.50,568122.50,17738.00,No Area Name Given,32.028555


In [20]:
narrowed_parcels_grouped.to_file("madison_taxes_per_sq_foot.geojson", driver='GeoJSON')

/Users/jameskreft/opt/anaconda3/envs/geopandas/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [19]:
#narrowed_parcels_grouped.explore("taxes_per_sq_foot", cmap="Blues",tooltip=False, popup=True, tiles='Stamen Toner' )

In [ ]:
from folium_glify_layer import GlifyLayer, Popup, Tooltip

for _, r in df.iterrows():
    # Without simplifying the representation of each borough,
    # the map might not be displayed
    sim_geo = gpd.GeoSeries(r['geometry']).simplify(tolerance=0.001)
    geo_j = sim_geo.to_json()
    geo_j = folium.GeoJson(data=geo_j,
                           style_function=lambda x: {'fillColor': 'orange'})
    folium.Popup(r['BoroName']).add_to(geo_j)
    geo_j.add_to(m)
m


# initialise a chroma js scale to be used for each feature
init_scale_function = """\
function(){
    return chroma.scale(chroma.brewer['viridis']).domain([0, 50]);
}
"""

# here we simply color each state using its index, u could
# easily do something more complex using feature.properties
color_feature_function = """\
function(index, feature){
    const color = scale(index)
      .rgba()
      .map(c => c / 255);
    return {
      r: color[0],
      g: color[1],
      b: color[2]
    };
}   
"""

# e.g.
popup = Popup({
    'fields': ["name"],
    'aliases': ["State"],
    'labels': True
})

tooltip = Tooltip({
    'fields': ["name"],
    'aliases': ["State"],
    'labels': True,
    'timeout_ms': 5000,
    'offset': [10, 10],
    'styles': {
        'background-color': '#CCC'
    }
})

layer = GlifyLayer(
    { 'shapes': geo_json_data }, 
    color_feature_function=color_feature_function, 
    init_scale_function=init_scale_function,
    glify_options={
        'border': False,
        'opacity': 1
    },
    tooltip=tooltip,
    # popup=popup
)

layer.add_to(m)
m